In [1]:
%matplotlib inline
import cv2
import numpy as np
import matplotlib.pyplot as plt
# from skimage import exposure
# from scipy.ndimage import gaussian_filter

import helpers

import pandas as pd
# import numpy as np
from PIL import Image
import PIL
import matplotlib as mpl

from skimage import exposure
from scipy import ndimage
from scipy.ndimage import gaussian_filter
from scipy.signal import find_peaks
from scipy.signal import argrelextrema

from datetime import datetime


from multiprocessing import Pool


In [3]:
filenames = !ls benoitdata/F20-14A/

In [4]:
def multi_figures(file):
    img = cv2.imread('benoitdata/F20-14A/'+file,0)
#     i = label_pipeline(img)
    i = helpers.label_pipeline(img)
#     np.save(arr=i, file='labeled/F20_14/'+file+'_labeled.npy')
    np.save(arr=i, file='/media/sda/data/labeled/F20_14/'+file+'_labeled.npy')
    fig, ax = plt.subplots(1, 2, figsize=(20, 10))

    ax[0].imshow(exposure.equalize_hist(img), cmap='plasma')
    ax[1].imshow(i, cmap='plasma')
    fig.tight_layout()
    fig.savefig('/media/sda/data/labeled/F20_14/'+file+'_labeled.png', dpi=300, bbox_inches='tight')
    plt.close()
    del img, i, fig, ax

In [ ]:
pool = Pool(maxtasksperchild=10)

start = datetime.now()
print('started at ', start)

pool.map(multi_figures, filenames)
pool.close()

print('this took', datetime.now()-start)

started at  2021-05-09 17:02:35.152568


In [ ]:
pool.close()

In [ ]:
# start = datetime.now()
# print('started at ', start)

# for file in filenames:
# #     file = 'benoitdata/F20_10_b/'+file
#     print(file)
# #     file = 'benoitdata/F20_10_b/6um_jamtveit_att_pag-1.99_3.12_00030.tif'

#     img = cv2.imread('benoitdata/F20_10_b/'+file,0)
#     i = label_pipeline(img)
#     np.savetxt(X=i, fname='labeled/'+file+'_labeled.dat', delimiter=',')
# #     np.savetxt(X=i, fname='labeled/6um_jamtveit_att_pag-1.99_3.12_00030.tif_labeled.dat', delimiter=',')
    
#     fig, ax = plt.subplots(1, 2, figsize=(20, 10))

#     ax[0].imshow(exposure.equalize_hist(img), cmap='plasma')
#     ax[1].imshow(i, cmap='plasma')
#     fig.tight_layout()
#     fig.savefig('labeled/'+file+'_labeled.png', dpi=300, bbox_inches='tight')
#     plt.close()
# #     fig.savefig('labeled/6um_jamtveit_att_pag-1.99_3.12_00030.tif_labeled.png', dpi=300, bbox_inches='tight')
# #     break

# print('this took', datetime.now()-start)

In [ ]:
# TODO:
# 1. multiprocess this
# 2. figure out splitter function
# 3. print out more figures to not make it so black box
# 4. go through all the figures created to see how good its doing

In [ ]:
# fig, ax = plt.subplots(figsize=(15, 15))

# ax.imshow(exposure.equalize_hist(img), cmap='Greys', alpha=0.5)
# # ax.imshow(i, alpha=0.5, cmap='plasma')

In [ ]:
# plt.imshow(i)

In [ ]:
# plt.imshow(img)

In [ ]:
# # filename = 'benoitdata/F20_10_b/6um_jamtveit_att_pag-1.99_3.12_00030.tif'
# filename = 'benoitdata/F20_10_b/6um_jamtveit_att_pag-1.99_3.12_00005.tif'

# img = cv2.imread(filename,0)
# # img = cv2.imread(file,0)


In [ ]:
# img

In [ ]:
# fig, ax = plt.subplots(1, 2, figsize=(20, 10))

# ax[0].imshow(exposure.equalize_hist(img), cmap='plasma')
# ax[1].imshow(zeros, cmap='plasma')
# fig.tight_layout()

In [ ]:
# zeros = label_pipeline(img)

In [ ]:
# fig, ax = plt.subplots(figsize=(20, 20))

# ax.imshow(zeros, cmap='inferno')